# 1. Thư viện

In [1]:
import os
import json
import re
from docx import Document

# 2. Thực thi

## 2.1. Hàm đếm từ

In [2]:
# Hàm đếm số tokens đơn giản dựa trên khoảng trắng
def count_tokens_simple(text):
    return len(text.split())

## 2.2. Hàm trích xuất các từ khóa

In [3]:
# Hàm trích xuất từ khóa từ văn bản, loại bỏ dấu câu ngoại trừ "/" và "-"
def extract_keywords(text):
    text = re.sub(r'[^\w\s/-]', '', text)
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

In [4]:
# Hàm xử lý các từ khóa: chuyển về chữ thường, giữ nguyên số La Mã và từ khóa có "/" hoặc "-"
def process_keywords(keywords):
    result = []
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường
    return result

## 2.3. Hàm xử lý từ khóa cho các key có giá trị không null

In [5]:
def process_value_to_keywords(value):
    if value is None or value.strip() == "":
        return None
    keywords = extract_keywords(value)
    return process_keywords(keywords)

## 2.4. Hàm xử lý tên file

In [6]:
def process_file_name(file_name):
    parts = file_name.replace('.docx', '').split('_')

    if len(parts) != 8:
        return None  # Nếu file không hợp lệ
    
    stt = parts[0]
    loai_van_ban = parts[1]
    noi_ban_hanh = parts[2]
    so_hieu = parts[3].replace("-", "/")
    linhvuc_nganh = parts[4]
    ngay_ban_hanh = parts[5].replace("-", "/")
    ngay_hieu_luc = parts[6] if parts[6] == "Đã biết" else parts[6].replace("-", "/")
    chu_de = parts[7]
    
    # Trích xuất và xử lý từ khóa từ file name
    metadata_keywords = extract_keywords(f"{loai_van_ban} {noi_ban_hanh} {so_hieu} {linhvuc_nganh} {ngay_ban_hanh} {ngay_hieu_luc} {chu_de}")
    metadata_keywords = process_keywords(metadata_keywords)

    return {
        "stt": stt,
        "loai_van_ban": loai_van_ban,
        "loai_van_ban_Keywords": process_value_to_keywords(loai_van_ban),
        "noi_ban_hanh": noi_ban_hanh,
        "noi_ban_hanh_Keywords": process_value_to_keywords(noi_ban_hanh),
        "so_hieu": so_hieu,
        "linhvuc_nganh": linhvuc_nganh,
        "linhvuc_nganh_Keywords": process_value_to_keywords(linhvuc_nganh),
        "ngay_ban_hanh": ngay_ban_hanh,
        "ngay_hieu_luc": ngay_hieu_luc,
        "chu_de": chu_de,
        "chu_de_Keywords": process_value_to_keywords(chu_de),
        "key_words": metadata_keywords  # Trả về từ khóa đã xử lý
    }

## 2.5. Hàm đọc và xử lý nội dung của file .docx

In [7]:
def process_docx(file_path, file_metadata, max_tokens=None):  
    document = Document(file_path)

    current_chapter = None
    current_section = None
    current_mini_section = None
    data = []
    
    for i in range(len(document.paragraphs)):
        paragraph_text = document.paragraphs[i].text.strip()

        # Bắt đầu một Chương mới
        if paragraph_text.startswith("Chương"):
            current_chapter = paragraph_text
            current_section = None
            current_mini_section = None
            if i + 1 < len(document.paragraphs):
                next_paragraph = document.paragraphs[i + 1].text.strip()
                if not next_paragraph.startswith(("Mục", "Tiểu mục", "Điều")):
                    current_chapter += f": {next_paragraph}"
            continue

        # Bắt đầu một Mục mới
        if paragraph_text.startswith("Mục"):
            current_section = paragraph_text
            current_mini_section = None
            continue

        # Bắt đầu một Tiểu mục mới
        if paragraph_text.startswith("Tiểu mục"):
            current_mini_section = paragraph_text
            continue

        # Bắt đầu một Điều mới
        if paragraph_text.startswith("Điều"):
            article = paragraph_text
            content = []
            article_sections = []

            # Tạo mảng A chứa các đoạn văn trong Điều
            A = []
            for j in range(i + 1, len(document.paragraphs)):
                next_paragraph = document.paragraphs[j].text.strip()

                # Nếu gặp Chương, Mục, Tiểu mục, Điều mới thì dừng lại
                if next_paragraph.startswith(("Chương", "Mục", "Tiểu mục", "Điều")):
                    break
                A.append(next_paragraph)
            
            # Tạo mảng B chứa các đoạn văn bắt đầu bằng số (Khoản)
            B = [para for para in A if re.match(r'^\d+\.', para)]

            # Đếm số Khoản (số đoạn bắt đầu bằng số)
            num_of_clauses = len(B)

            # Duyệt qua từng Khoản (nếu có)
            if num_of_clauses > 0:
                for clause_index in range(1, num_of_clauses + 1):
                    section_content = []
                    found_clause = False

                    # Duyệt qua mảng A để trích xuất nội dung của từng Khoản
                    for para in A:
                        # Nếu gặp Khoản mới (đoạn bắt đầu bằng số)
                        if re.match(rf'^{clause_index}\.', para):
                            found_clause = True
                            section_content.append(para)  # Lưu lại đoạn văn của Khoản
                        elif found_clause and re.match(r'^\d+\.', para):
                            # Nếu gặp đoạn văn bắt đầu bằng số tiếp theo (Khoản mới) thì dừng lại
                            break
                        elif found_clause:
                            section_content.append(para)  # Lưu nội dung các đoạn văn trong Khoản

                    # Lưu dữ liệu của Khoản
                    section_text = "\n".join(section_content).strip()

                    # Tạo từ khóa bao gồm các trường "Chapter", "Section", "Mini-Section", và "Article-Section"
                    structural_info = f"{current_chapter} {current_section or ''} {current_mini_section or ''} {f'Khoản {clause_index}'}"
                    content_key_words = process_keywords(extract_keywords(f"{article}\n{section_text} {structural_info}"))
                    all_key_words = list(dict.fromkeys(file_metadata["key_words"] + content_key_words))

                    # Xử lý từ khóa có ký tự "/"
                    for kw in all_key_words[:]:
                        if "/" in kw:
                            new_kw = kw.replace("/", "-")
                            if new_kw not in all_key_words:
                                all_key_words.append(new_kw)

                    noiDung_Khoan = f"Khoản {clause_index}"

                    # Tạo trường "combine_Article_Content"
                    combine_article_content = f"{article}\n{section_text}"
                    combine_article_content_keywords = process_keywords(extract_keywords(combine_article_content))

                    # Lưu dữ liệu Khoản vào mảng data
                    data.append({
                        "Chapter": current_chapter,
                        "Chapter_Keywords": process_value_to_keywords(current_chapter),
                        "Section": current_section if current_section else None,
                        "Section_Keywords": process_value_to_keywords(current_section),
                        "Mini-Section": current_mini_section if current_mini_section else None,
                        "Mini-Section_Keywords": process_value_to_keywords(current_mini_section),
                        "Article": article,
                        "Article_Keywords": process_value_to_keywords(article),
                        "Content": section_text,  # Nội dung của Khoản
                        "Content_Keywords": process_value_to_keywords(section_text),
                        "Article-Section": noiDung_Khoan,  # Loại bỏ dấu chấm
                        "Article-Section_Keywords": process_value_to_keywords(noiDung_Khoan),
                        "combine_Article_Content": combine_article_content,
                        "combine_Article_Content_Keywords": combine_article_content_keywords,  # Từ khóa cho combine_Article_Content
                        "key_words": all_key_words
                    })
            else:
                # Trường hợp không có Khoản
                content_text = "\n".join(A).strip()

                # Tạo từ khóa bao gồm các trường "Chapter", "Section", "Mini-Section", và "Article-Section"
                structural_info = f"{current_chapter} {current_section or ''} {current_mini_section or ''}"
                content_key_words = process_keywords(extract_keywords(f"{article}\n{content_text} {structural_info}"))
                all_key_words = list(dict.fromkeys(file_metadata["key_words"] + content_key_words))

                # Xử lý từ khóa có ký tự "/"
                for kw in all_key_words[:]:
                    if "/" in kw:
                        new_kw = kw.replace("/", "-")
                        if new_kw not in all_key_words:
                            all_key_words.append(new_kw)

                # Tạo trường "combine_Article_Content"
                combine_article_content = f"{article}\n{content_text}"
                combine_article_content_keywords = process_keywords(extract_keywords(combine_article_content))

                # Lưu dữ liệu Điều không có Khoản
                data.append({
                    "Chapter": current_chapter,
                    "Chapter_Keywords": process_value_to_keywords(current_chapter),
                    "Section": current_section if current_section else None,
                    "Section_Keywords": process_value_to_keywords(current_section),
                    "Mini-Section": current_mini_section if current_mini_section else None,
                    "Mini-Section_Keywords": process_value_to_keywords(current_mini_section),
                    "Article": article,
                    "Article_Keywords": process_value_to_keywords(article),
                    "Content": content_text,
                    "Content_Keywords": process_value_to_keywords(content_text),
                    "Article-Section": None,
                    "Article-Section_Keywords": None,
                    "combine_Article_Content": combine_article_content,
                    "combine_Article_Content_Keywords": combine_article_content_keywords,  # Từ khóa cho combine_Article_Content
                    "key_words": all_key_words
                })

    return data

## 2.6. Hàm chính xử lý tất cả các file trong thư mục

In [8]:
def process_folder(folder_path, output_json_path, max_tokens):
    all_data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".docx"):
            file_metadata = process_file_name(file_name)
            if file_metadata is None:
                continue
            
            file_path = os.path.join(folder_path, file_name)
            docx_data = process_docx(file_path, file_metadata, max_tokens=max_tokens)
            
            for entry in docx_data:
                combined_entry = {**file_metadata, **entry}
                all_data.append(combined_entry)
    
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False, indent=4)
    
    print(f"Dữ liệu đã được lưu vào file {output_json_path}")

## 2.6. Thực thi

In [9]:
# Đường dẫn tới thư mục và file đầu ra
folder_path = r'data\trich_dan_luat\docx\Luat'
json_file_path = r'data\trich_dan_luat\json\demo_Article-Section_Combined_Output_More_Keywords.json'
max_tokens = None  # Giới hạn số token

In [10]:
# Chạy chương trình chính
process_folder(folder_path, json_file_path, max_tokens)

Dữ liệu đã được lưu vào file data\trich_dan_luat\json\demo_Article-Section_Combined_Output_More_Keywords.json
